## Sorry for large amount of junk

In [1]:
import pandas as pd
import numpy as np

In [2]:
import gensim

Using TensorFlow backend.


In [16]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.externals import joblib

import numpy as np

In [44]:
bayes_clf = Pipeline([('vect', CountVectorizer(encoding='koir8')),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB())
                      ])
bayes_clf.fit(train.comment, train.reting.astype('uint8'))

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='koir8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [45]:
res = bayes_clf.predict(test.comment)

In [91]:


import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

from keras.layers import Input, Dense, Embedding, merge, Convolution2D, MaxPooling2D, Dropout
from sklearn.cross_validation import train_test_split
from keras.layers.core import Reshape, Flatten
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model

In [35]:
top_words = 5000

In [51]:

max_words = 1000
batch_size = 32
epochs = 5


In [106]:
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(X_test, mode='binary')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [112]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 11690 samples, validate on 3897 samples
Epoch 1/3
11690/11690 [==============================] - 78s - loss: -29.4665 - acc: 0.0541 - val_loss: -33.6181 - val_acc: 0.0570
Epoch 2/3
11690/11690 [==============================] - 77s - loss: -33.8258 - acc: 0.0541 - val_loss: -33.6181 - val_acc: 0.0570
Epoch 3/3
11690/11690 [==============================] - 77s - loss: -33.8258 - acc:

In [113]:
sequence_length = 45
embedding_dim = 20
filter_sizes = (3, 4)
num_filters = 128
dropout_prob = (0.25, 0.5)
hidden_dims = 128 

# Training parameters
batch_size = 32
num_epochs = 30
val_split = 0.1

# Word2Vec parameters, see train_word2vec
min_word_count = 1  # Minimum word count
context = 10        # Context window size

In [117]:
tr_comment = []
for comment in train.comment:
    tr_comment.append(comment.replace(/[^a-z\u0400-\u04FF]/gi," "))

SyntaxError: invalid syntax (<ipython-input-117-ac58f4d62897>, line 3)

In [114]:
tfidf = TfidfTransformer()

In [109]:
cv = CountVectorizer(max_features=500)
ft = tfidf.fit_transform(cv.fit_transform(train.comment))

In [116]:
t_ft = tfidf.fit_transform(cv.fit_transform(test.comment))

In [111]:
X_train = ft.toarray()
X_test = t_ft.toarray()
y_train = train.reting-1
y_test = test.reting-1

In [77]:
import xgboost as xgb

/home/dev/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [78]:
cls = xgb.XGBClassifier()

In [94]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [118]:
sequence_length = X_train.shape[1]
vocabulary_size = 200
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

nb_epoch = 100
batch_size = 30

# this returns a tensor
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(output_dim=embedding_dim, input_dim=vocabulary_size, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Convolution2D(num_filters, filter_sizes[0], embedding_dim, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(reshape)
conv_1 = Convolution2D(num_filters, filter_sizes[1], embedding_dim, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(reshape)
conv_2 = Convolution2D(num_filters, filter_sizes[2], embedding_dim, border_mode='valid', init='normal', activation='relu', dim_ordering='tf')(reshape)

maxpool_0 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_0)
maxpool_1 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_1)
maxpool_2 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), border_mode='valid', dim_ordering='tf')(conv_2)

merged_tensor = merge([maxpool_0, maxpool_1, maxpool_2], mode='concat', concat_axis=1)
flatten = Flatten()(merged_tensor)
# reshape = Reshape((3*num_filters,))(merged_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(output_dim=2, activation='softmax')(dropout)

# this creates a model that includes
model = Model(input=inputs, output=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, callbacks=[checkpoint], validation_data=(X_test, y_test))  # starts training


/home/dev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 256), activation="relu", padding="valid", data_format="channels_last", kernel_initializer="normal")`
  app.launch_new_instance()
/home/dev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (4, 256), activation="relu", padding="valid", data_format="channels_last", kernel_initializer="normal")`
/home/dev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (5, 256), activation="relu", padding="valid", data_format="channels_last", kernel_initializer="normal")`
/home/dev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(498, 1), strides=(1, 1), padding="valid", data_format="

ValueError: Error when checking target: expected dense_10 to have shape (None, 2) but got array with shape (11690, 1)

In [88]:

param = {'silent':1, 'objective':'multi:softmax' }
param['nthread'] = 8
param['eval_metric'] = 'merror'
param['eta'] = 0.075
param['max_depth'] = 20
param['silent'] = 1
param['num_class'] = 5
param['n_estimators'] = 200
param['early_stopping_rounds'] = 30

watchlist = [ (dtrain,'train'), (dtest, 'test') ]

bst = xgb.train(param, dtrain, 1500, watchlist,verbose_eval=25)

[0]	train-merror:0.280325	test-merror:0.474211
[25]	train-merror:0.18349	test-merror:0.442392
[50]	train-merror:0.138409	test-merror:0.43572
[75]	train-merror:0.104962	test-merror:0.434693
[100]	train-merror:0.086056	test-merror:0.432127
[125]	train-merror:0.073567	test-merror:0.434693
[150]	train-merror:0.065954	test-merror:0.437003
[175]	train-merror:0.059453	test-merror:0.439312
[200]	train-merror:0.055175	test-merror:0.443931
[225]	train-merror:0.050556	test-merror:0.446754
[250]	train-merror:0.046792	test-merror:0.448294
[275]	train-merror:0.044568	test-merror:0.449577
[300]	train-merror:0.042943	test-merror:0.450603
[325]	train-merror:0.040376	test-merror:0.451629
[350]	train-merror:0.038751	test-merror:0.452399
[375]	train-merror:0.037382	test-merror:0.45086
[400]	train-merror:0.035928	test-merror:0.448807
[425]	train-merror:0.034303	test-merror:0.44778
[450]	train-merror:0.033191	test-merror:0.449833
[475]	train-merror:0.03225	test-merror:0.449063
[500]	train-merror:0.031565	te

In [4]:
from sklearn.model_selection import train_test_split

## Acquiring data

In [120]:
train = pd.read_csv('X_train.csv', encoding='utf-8')

## Splitting data

In [121]:
train, test = train_test_split(train)

In [19]:
train = train[1:3000]
test= test[1:100]